In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter


## Data Preprocessing    

In [56]:
# get data from s3
url = "https://s3.amazonaws.com/survival2024/hurricane.csv"
df = pd.read_csv(url)

In [57]:
#check for motor failure
df['motor']=0
df.loc[df['reason'] == 2, 'motor'] = 1

#grab hour columns but first rename hour column
df = df.rename(columns={'hour': 'failure_hour'})
hour_columns = [col for col in df.columns if col.startswith('h')]

#fill na's in the hour_columns with 0's, I'm assuming that since they are missing they are not turned on
df[hour_columns] = df[hour_columns].fillna(0)

#add pump numbers to df
df = df.reset_index(names='pump')

#melt df to reshape it
df_long = df.melt(
    id_vars=[col for col in df.columns if col not in hour_columns],  # Static columns to keep
    value_vars=hour_columns,  # Columns to unpivot
    var_name='run_hour',  # New column name for hour labels
    value_name='running'  # Values from the hour columns
)

#grab numeric hour
df_long['run_hour'] = df_long['run_hour'].str.extract(r'(\d+)').astype(float).fillna(-1).astype(int)

#create a continous runtime column. how many hours has runnning=0 in a row
df_long['continuous_runtime'] = (
    df_long.groupby('pump')['running']
    .cumsum() * df_long['running']
)

df_long = df_long.sort_values(['pump', 'run_hour']).reset_index(drop=True)
df_long.head()


,pump,backup,age,bridgecrane,servo,gear,trashrack,slope,elevation,survive,failure_hour,reason,reason2,motor,run_hour,running,continuous_runtime
0,0,0,6.0,0,0,0,1,3,2,1,48,0,5,0,1,1.0,1.0
1,0,0,6.0,0,0,0,1,3,2,1,48,0,5,0,2,1.0,2.0
2,0,0,6.0,0,0,0,1,3,2,1,48,0,5,0,3,1.0,3.0
3,0,0,6.0,0,0,0,1,3,2,1,48,0,5,0,4,1.0,4.0
4,0,0,6.0,0,0,0,1,3,2,1,48,0,5,0,5,1.0,5.0


Now I'm going to grab only the columns in which I need to predict

In [58]:
df_motor = df_long[['pump', 'backup', 'age', 'bridgecrane', 'servo', 'gear', 'trashrack', 'slope', 'elevation', 'failure_hour', 'motor', 'run_hour', 'running', 'continuous_runtime']]
df_motor.head()

,pump,backup,age,bridgecrane,servo,gear,trashrack,slope,elevation,failure_hour,motor,run_hour,running,continuous_runtime
0,0,0,6.0,0,0,0,1,3,2,48,0,1,1.0,1.0
1,0,0,6.0,0,0,0,1,3,2,48,0,2,1.0,2.0
2,0,0,6.0,0,0,0,1,3,2,48,0,3,1.0,3.0
3,0,0,6.0,0,0,0,1,3,2,48,0,4,1.0,4.0
4,0,0,6.0,0,0,0,1,3,2,48,0,5,1.0,5.0


## Cox regression

In [59]:
cph = CoxPHFitter()
cph.fit(df_motor, duration_col='failure_hour', event_col='motor')
cph.print_summary()

c:\anaconda\envs\ds\Lib\site-packages\lifelines\utils\__init__.py:1120: ConvergenceWarning: Column trashrack have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['motor'].astype(bool)
>>> print(df.loc[events, 'trashrack'].var())
>>> print(df.loc[~events, 'trashrack'].var())

A very low variance means that the column trashrack completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
c:\anaconda\envs\ds\Lib\site-packages\lifelines\fitters\coxph_fitter.py:1607: ConvergenceWarning: Newton-Raphson convergence completed successfully but norm(delta) is still high, 0.500. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation 

<lifelines.CoxPHFitter: fitted with 36960 total observations, 31584 right-censored observations>
             duration col = 'failure_hour'
                event col = 'motor'
      baseline estimation = breslow
   number of observations = 36960
number of events observed = 5376
   partial log-likelihood = -43091.32
         time fit was run = 2024-11-18 00:49:15 UTC

---
                     coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                            
pump                 0.01      1.01      0.00            0.01            0.01                1.01                1.01
backup              -0.09      0.91      0.03           -0.15           -0.04                0.86                0.96
age                 -1.06      0.35      0.02           -1.10           -1.03                0.33                0.36
bridgecrane         -0.45      0.64      0.04           -0.53           -0.37                0.59                0.69
servo                0.23      1.25      0.03            0.16            0.29                1.18                1.34
gear                -0.12      0.88      0.05           -0.22           -0.03                0.80                0.97
trashrack          -18.67      0.00    256.17         -520.75          483.41                0.00           8.75e+209
slope               -0.25      0.78      0.01           -0.27           -0.23                0.77                0.80
elevation            0.12      1.12      0.02            0.08            0.15                1.09                1.16
run_hour             0.00      1.00      0.00           -0.00            0.00                1.00                1.00
running             -0.10      0.90      0.04           -0.19           -0.02                0.83                0.98
continuous_runtime  -0.00      1.00      0.00           -0.01           -0.00                0.99                1.00

                    cmp to      z      p  -log2(p)
covariate                                         
pump                  0.00  60.62 <0.005       inf
backup                0.00  -3.22 <0.005      9.59
age                   0.00 -57.15 <0.005       inf
bridgecrane           0.00 -10.63 <0.005     85.22
servo                 0.00   6.95 <0.005     37.99
gear                  0.00  -2.49   0.01      6.30
trashrack             0.00  -0.07   0.94      0.09
slope                 0.00 -23.29 <0.005    396.11
elevation             0.00   6.55 <0.005     34.00
run_hour              0.00   1.55   0.12      3.04
running               0.00  -2.39   0.02      5.90
continuous_runtime    0.00  -2.00   0.05      4.45
---
Concordance = 0.94
Partial AIC = 86206.64
log-likelihood ratio test = 20736.73 on 12 df
-log2(p) of ll-ratio test = inf